# Training and Evaluation

In [6]:
import cv2 as cv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms


from torch.utils.data import DataLoader
from models.baseline_average import BaselineAverageModel
from models.convolved_average import ConvolvedAverageModel
from models.empty_checkerboard import EmptyCheckerboardModel
from reconstruct import nparray_to_mp4, mp4_to_nparray
from score import calculate_mse
from models.video_dataset import VideoDataset


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
IN_HEIGHT = 2160
IN_WIDTH = 1920
OUT_HEIGHT = 2160
OUT_WIDTH = 3840

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.is_available()

True

In [8]:
# input_path = "data/Seconds_That_Count-checkerboard.mov"
# output_path_cam = "data/Seconds_That_Count-cvavg.mov"
# output_path_bavg = "data/Seconds_That_Count-bavg.mov"
# native_path = "data\Seconds_That_Count-native.mov"
# empty_checkerboard_path = "data\LifeSeconds_That_Count-checkerboard-empty.mov"

# input_path = "data\Lifting_Off-checkerboard.mov"
# output_path_cam = "data/Lifting_Off-cvavg.mov"
# output_path_bavg = "data/Lifting_Off-bavg.mov"
# native_path = "data\Lifting_Off-native.mov"

# input_path = "data\Moment+of+Intensity-checkerboard.mov"
# output_path_cam = "data/Moment+of+Intensity-cvavg.mov"
# output_path_bavg = "data/Moment+of+Intensity-bavg.mov"
# native_path = "data\Moment+of+Intensity-native.mov"

input_path = "data\LifeUntouched_P3_4K_PQ_XQ-checkerboard.mov"
output_path_cam = "data/LifeUntouched_P3_4K_PQ_XQ-cvavg.mov"
output_path_bavg = "data/LifeUntouched_P3_4K_PQ_XQ-bavg.mov"
empty_checkerboard_path = "data\LifeUntouched_P3_4K_PQ_XQ-checkerboard-empty.mov"
output_path_s3dcnn = "data/LifeUntouched_P3_4K_PQ_XQ-s3dcnn.mov"
native_path = "data\LifeUntouched_P3_4K_PQ_XQ-native.mov"

In [4]:
# Instantiate model
cavg_model = ConvolvedAverageModel(in_shape=(IN_HEIGHT, IN_WIDTH), out_shape=(OUT_HEIGHT, OUT_WIDTH))

In [10]:
cavg_model(input_path, output_path_cam)

Upscaled video saved to data/LifeUntouched_P3_4K_PQ_XQ-cvavg.mov


In [7]:
bavg_model = BaselineAverageModel(in_shape=(IN_HEIGHT, IN_WIDTH), out_shape=(OUT_HEIGHT, OUT_WIDTH))

In [11]:
bavg_model(input_path, output_path_bavg)

Finished processing video.


In [ ]:
# evaluate
calculate_mse(native_path, output_path_bavg)


In [ ]:
calculate_mse(native_path, output_path_cam)


In [ ]:
empty_checkerboard = EmptyCheckerboardModel(in_shape=(IN_HEIGHT, IN_WIDTH), out_shape=(OUT_HEIGHT, OUT_WIDTH))

In [ ]:
empty_checkerboard(input_path, empty_checkerboard_path)

### **FSR-CNN**

In [ ]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\Seconds_That_Count-1080p.mov", "data\Seconds_That_Count-native.mov", epoches=20, lr=0.01, max_frames=50)

In [2]:
from models.cnn import CNNReconstructor

cnn_reconstructor = CNNReconstructor("saved_epoches\srcnn_best_seconds_that_count.pth")
cnn_reconstructor.process_video("data\Seconds_That_Count-1080p.mov", "Seconds_That_Count_srcnn_.mov")

3840 2160
Reconstructed video saved to Seconds_That_Count_srcnn_.mov


In [4]:
cnn_reconstructor = CNNReconstructor("srcnn_best.pth")
cnn_reconstructor.process_video("data\Seconds_That_Count-1080p.mov", "Seconds_That_Count_srcnn_1.mov")

3840 2160
Reconstructed video saved to Seconds_That_Count_srcnn_1.mov


In [5]:
calculate_mse("data\Seconds_That_Count-native.mov", "Seconds_That_Count_srcnn_.mov")

np.float32(10999471000.0)

In [7]:
calculate_mse("data\Seconds_That_Count-native.mov", "Seconds_That_Count_srcnn_1.mov")

np.float32(6683265000.0)

In [2]:
from models.cnn import CNNReconstructor

cnn_reconstructor = CNNReconstructor("srcnn_best.pth")
cnn_reconstructor.process_video("data\Lifting_Off-1080p.mov", "Lifting_Off_srcnn_1.mov")

3840 2160
Reconstructed video saved to Lifting_Off_srcnn_1.mov


In [ ]:
trainSuperResCNN("data\Lifting_Off-1080p.mov", "data\Lifting_Off-native.mov", epoches=20, lr=0.001, max_frames=50)

In [1]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\ImageDataset1080p_chunk_3.mov", "data\ImageDataset4K_chunk_3.mov", epoches=20, lr=0.001, max_frames=100, pretrained="srcnn_epoch_6.pth")

Begin Training
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Epoch 0, Loss: 1049.051785888672, PSNR: -29.178693771362305
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Epoch 1, Loss: 968.22740234375, PSNR: -28.89354133605957
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98

OutOfMemoryError: CUDA out of memory. Tried to allocate 508.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 25.70 GiB is allocated by PyTorch, and 275.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\ImageDataset1080p_chunk_1.mov", "data\ImageDataset4K_chunk_1.mov", epoches=20, lr=0.001, max_frames=10, pretrained=None)

c:\Users\harvi\miniconda3\envs\cs229\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Begin Training
frame: 90
frame: 10
frame: 50
frame: 30
frame: 70
frame: 0
frame: 80
frame: 60
frame: 20
frame: 40
Epoch 0, Loss: 17689.6005859375, PSNR: -41.68474197387695
frame: 61
frame: 41
frame: 11
frame: 31
frame: 51
frame: 21
frame: 1
frame: 81
frame: 71
Epoch 1, Loss: 17814.913628472223, PSNR: -41.65751266479492
frame: 72
frame: 52
frame: 22
frame: 2
frame: 42
frame: 62
frame: 12
frame: 32
frame: 82
Epoch 2, Loss: 15873.41650390625, PSNR: -40.8039436340332
frame: 43
frame: 33
frame: 3
frame: 23
frame: 13
frame: 73
frame: 53
frame: 63
frame: 83
Epoch 3, Loss: 7214.046549479167, PSNR: -37.43090057373047
frame: 64
frame: 44
frame: 84
frame: 4
frame: 24
frame: 74
frame: 34
frame: 54
frame: 14
Epoch 4, Loss: 14258.233303493924, PSNR: -40.15651321411133
frame: 45
frame: 35
frame: 25
frame: 75
frame: 85
frame: 5
frame: 55
frame: 65
frame: 15
Epoch 5, Loss: 9778.648980034723, PSNR: -37.82392120361328
frame: 86
frame: 56
frame: 76
frame: 36
frame: 26
frame: 66
frame: 6
frame: 16
frame: 4

RuntimeError: cuDNN error: CUDNN_STATUS_INTERNAL_ERROR_HOST_ALLOCATION_FAILED

In [ ]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\ImageDataset1080p_chunk_2.mov", "data\ImageDataset4K_chunk_2.mov", epoches=20, lr=0.001, max_frames=10, pretrained="saved_epoches/srcnn_epoch_12_vgg.pth")

c:\Users\harvi\miniconda3\envs\cs229\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Begin Training
frame: 60
frame: 30
frame: 50
frame: 10
frame: 20
frame: 40
frame: 90
frame: 70
frame: 0
frame: 80
Epoch 0, Loss: 3880.5590698242186, PSNR: -33.64617919921875
frame: 81
frame: 11
frame: 71
frame: 31
frame: 41
frame: 61
frame: 1
frame: 51
frame: 21
Epoch 1, Loss: 5635.796196831598, PSNR: -35.53208923339844
frame: 52
frame: 72
frame: 82
frame: 2
frame: 22
frame: 62
frame: 12
frame: 32
frame: 42
Epoch 2, Loss: 4113.685085720486, PSNR: -34.72306442260742
frame: 13
frame: 53
frame: 43
frame: 73
frame: 63
frame: 83
frame: 3
frame: 23
frame: 33
Epoch 3, Loss: 3868.8960774739585, PSNR: -34.306026458740234
frame: 44
frame: 24
frame: 4
frame: 64
frame: 14
frame: 54
frame: 74
frame: 84
frame: 34
Epoch 4, Loss: 4136.11956108941, PSNR: -34.2485466003418
frame: 5
frame: 85
frame: 35
frame: 25
frame: 15
frame: 45
frame: 65
frame: 55
frame: 75
Epoch 5, Loss: 4232.506944444444, PSNR: -34.06157302856445
frame: 56
frame: 66
frame: 86
frame: 76
frame: 36
frame: 6
frame: 16
frame: 46
frame: 

In [1]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\ImageDataset1080p_chunk_3.mov", "data\ImageDataset4K_chunk_3.mov", epoches=10, lr=0.001, max_frames=100, pretrained="saved_epoches/srcnn_epoch_10_vgg_chunk2.pth")

c:\Users\harvi\miniconda3\envs\cs229\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Begin Training
frame: 40
frame: 90
frame: 30
frame: 10
frame: 70
frame: 80
frame: 20
frame: 60
frame: 0
frame: 50
Epoch 0, Loss: 3605.1386108398438, PSNR: -33.05524444580078
frame: 61
frame: 71
frame: 51
frame: 11
frame: 31
frame: 1
frame: 21
frame: 41
frame: 81
Epoch 1, Loss: 2778.228800455729, PSNR: -31.649946212768555
frame: 22
frame: 62
frame: 12
frame: 52
frame: 82
frame: 42
frame: 2
frame: 72
frame: 32
Epoch 2, Loss: 3734.7481146918403, PSNR: -32.69332504272461
frame: 3
frame: 23
frame: 83
frame: 33
frame: 63
frame: 53
frame: 13
frame: 43
frame: 73
Epoch 3, Loss: 3760.262166341146, PSNR: -32.46623611450195
frame: 4
frame: 64
frame: 24
frame: 34
frame: 44
frame: 84
frame: 14
frame: 54
frame: 74
Epoch 4, Loss: 3905.578870985243, PSNR: -33.5766487121582
frame: 5
frame: 15
frame: 65
frame: 35
frame: 25
frame: 55
frame: 85
frame: 75
frame: 45
Epoch 5, Loss: 3338.1923149956597, PSNR: -32.97666931152344
frame: 86
frame: 76
frame: 46
frame: 36
frame: 16
frame: 6
frame: 56
frame: 26
frame

In [1]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\ImageDataset1080p_chunk_1.mov", "data\ImageDataset4K_chunk_1.mov", epoches=20, lr=0.001, max_frames=100, pretrained=None)

Begin Training
frame: 20
frame: 60
frame: 70
frame: 10
frame: 50
frame: 30
frame: 40
frame: 80
frame: 90
frame: 0
Epoch 0, Loss: 0.15369411781430245, PSNR: 8.67150592803955
frame: 31
frame: 81
frame: 21
frame: 11
frame: 41
frame: 71
frame: 1
frame: 51
frame: 61
Epoch 1, Loss: 0.10786363151338366, PSNR: 9.823347091674805
frame: 52
frame: 62
frame: 72
frame: 82
frame: 22
frame: 2
frame: 32
frame: 12
frame: 42
Epoch 2, Loss: 0.09705405020051533, PSNR: 10.315824508666992
frame: 73
frame: 13
frame: 43
frame: 83
frame: 53
frame: 33
frame: 23
frame: 63
frame: 3
Epoch 3, Loss: 0.0496564581990242, PSNR: 13.160520553588867
frame: 34
frame: 24
frame: 74
frame: 64
frame: 54
frame: 4
frame: 44
frame: 14
frame: 84
Epoch 4, Loss: 0.08905112453632885, PSNR: 11.26262092590332
frame: 55
frame: 35
frame: 15
frame: 45
frame: 75
frame: 85
frame: 65
frame: 25
frame: 5
Epoch 5, Loss: 0.0630111104498307, PSNR: 12.680047035217285
frame: 76
frame: 6
frame: 36
frame: 26
frame: 66
frame: 86
frame: 46
frame: 16
fr

In [1]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\ImageDataset1080p.mov", "data\ImageDataset4K.mov", epoches=105, lr=0.001, max_frames=350, pretrained=None)

Begin Training
frame: 175
frame: 140
frame: 210
frame: 315
frame: 245
frame: 105
frame: 70
frame: 0
frame: 35
frame: 280
Epoch 0, Loss: 0.20354339033365249, PSNR: 7.095168590545654
frame: 176
frame: 281
frame: 246
frame: 211
frame: 141
frame: 36
frame: 1
frame: 71
frame: 106
Epoch 1, Loss: 0.09901963671048482, PSNR: 10.363152503967285
frame: 247
frame: 37
frame: 72
frame: 107
frame: 142
frame: 282
frame: 177
frame: 2
frame: 212
Epoch 2, Loss: 0.07245230633351538, PSNR: 11.57144832611084
frame: 143
frame: 283
frame: 3
frame: 178
frame: 38
frame: 248
frame: 108
frame: 213
frame: 73
Epoch 3, Loss: 0.05727443678511514, PSNR: 12.75184440612793
frame: 144
frame: 39
frame: 109
frame: 74
frame: 284
frame: 4
frame: 179
frame: 214
frame: 249
Epoch 4, Loss: 0.04158857588966688, PSNR: 13.922107696533203
frame: 145
frame: 40
frame: 110
frame: 75
frame: 285
frame: 215
frame: 5
frame: 180
frame: 250
Epoch 5, Loss: 0.04775244721935855, PSNR: 13.737640380859375
frame: 6
frame: 286
frame: 181
frame: 76


In [1]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\ImageDataset1080p.mov", "data\ImageDataset4K.mov", epoches=1000, lr=0.001, max_frames=350, pretrained="srcnn_epoch_104.pth")

Begin Training
Epoch 0, Loss: 0.061203960329294205, PSNR: 12.493861198425293
Epoch 1, Loss: 0.037067668305502996, PSNR: 14.69542407989502
Epoch 2, Loss: 0.027380264467663236, PSNR: 15.907121658325195
Epoch 3, Loss: 0.026893960415489145, PSNR: 16.317659378051758
Epoch 4, Loss: 0.02098725436048375, PSNR: 17.051288604736328
Epoch 5, Loss: 0.035116896447208196, PSNR: 15.40030288696289
Epoch 6, Loss: 0.027464873467882473, PSNR: 16.02372932434082
Epoch 7, Loss: 0.03239831804401345, PSNR: 15.152801513671875
Epoch 8, Loss: 0.027388634263641305, PSNR: 16.011859893798828
Epoch 9, Loss: 0.02661830900857846, PSNR: 16.074813842773438
Epoch 10, Loss: 0.019658539754649002, PSNR: 17.567033767700195
Epoch 11, Loss: 0.019584459770056937, PSNR: 17.29256248474121
Epoch 12, Loss: 0.019627633711530104, PSNR: 17.726333618164062
Epoch 13, Loss: 0.02895136870857742, PSNR: 15.83260440826416
Epoch 14, Loss: 0.032273366736869015, PSNR: 15.175564765930176
Epoch 15, Loss: 0.0250818133354187, PSNR: 16.31143951416015

In [1]:
from models.cnn import trainSuperResCNN
# clipping
trainSuperResCNN("data\ImageDataset1080p.mov", "data\ImageDataset4K.mov", epoches=1000, lr=0.001, max_frames=350, pretrained=None)

Begin Training
Epoch 0, Loss: 0.11600397489964961, PSNR: 9.962438583374023
Epoch 1, Loss: 0.10637997421953413, PSNR: 10.2571382522583
Epoch 2, Loss: 0.05956442943877644, PSNR: 12.451992988586426
Epoch 3, Loss: 0.06817385140392515, PSNR: 11.967613220214844
Epoch 4, Loss: 0.05230171171327432, PSNR: 13.022807121276855
Epoch 5, Loss: 0.05690378612942166, PSNR: 12.774652481079102
Epoch 6, Loss: 0.051580491993162364, PSNR: 13.241263389587402
Epoch 7, Loss: 0.0451219342648983, PSNR: 13.769376754760742
Epoch 8, Loss: 0.04884077153272099, PSNR: 13.441363334655762
Epoch 9, Loss: 0.04397738021281031, PSNR: 14.17607307434082
Epoch 10, Loss: 0.03530065777401129, PSNR: 14.89384937286377
Epoch 11, Loss: 0.03522435503287448, PSNR: 14.963263511657715
Epoch 12, Loss: 0.03369252093964153, PSNR: 15.077420234680176
Epoch 13, Loss: 0.04006675278974904, PSNR: 14.440421104431152
Epoch 14, Loss: 0.047359676410754524, PSNR: 13.827439308166504
Epoch 15, Loss: 0.037908723991778165, PSNR: 14.617984771728516
Epoch 

In [3]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\Seconds_That_Count-1080p.mov", "data\Seconds_That_Count-native.mov", epoches=101, lr=0.001, max_frames=500, pretrained="saved_epoches/srcnn_best_norm_1k.pth")

Begin Training
Epoch 0, Loss: 0.021732498565688728, PSNR: 17.281017303466797
Epoch 1, Loss: 0.026227229067848787, PSNR: 18.836938858032227
Epoch 2, Loss: 0.017459154180768464, PSNR: 19.323936462402344
Epoch 3, Loss: 0.02078454887184004, PSNR: 21.713459014892578
Epoch 4, Loss: 0.012929585658841662, PSNR: 21.046310424804688
Epoch 5, Loss: 0.010028051225365035, PSNR: 23.12824249267578
Epoch 6, Loss: 0.007670663724032541, PSNR: 24.5892276763916
Epoch 7, Loss: 0.006639341927236981, PSNR: 24.670827865600586
Epoch 8, Loss: 0.01095373310898948, PSNR: 23.962905883789062
Epoch 9, Loss: 0.01017018351962583, PSNR: 23.65360450744629
Epoch 10, Loss: 0.006016725855362084, PSNR: 24.473297119140625
Epoch 11, Loss: 0.009581628216740986, PSNR: 23.07928466796875
Epoch 12, Loss: 0.008230741989488402, PSNR: 22.956010818481445
Epoch 13, Loss: 0.007012779572202514, PSNR: 24.085826873779297
Epoch 14, Loss: 0.004208020309710668, PSNR: 24.833959579467773
Epoch 15, Loss: 0.005569940251815651, PSNR: 24.62086868286

In [1]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\Seconds_That_Count-1080p.mov", "data\Seconds_That_Count-native.mov", epoches=101, lr=0.001, max_frames=500, pretrained="srcnn_best.pth")

Begin Training
Epoch 0, Loss: 0.06001474615186453, PSNR: 13.904739379882812
Epoch 1, Loss: 0.0411764116336902, PSNR: 17.599254608154297
Epoch 2, Loss: 0.02637860647195743, PSNR: 18.521806716918945
Epoch 3, Loss: 0.019918369103430048, PSNR: 19.9708194732666
Epoch 4, Loss: 0.016452649750539824, PSNR: 21.06248664855957
Epoch 5, Loss: 0.014031769216267599, PSNR: 21.919532775878906
Epoch 6, Loss: 0.013208726071752608, PSNR: 22.259456634521484
Epoch 7, Loss: 0.010897765427620875, PSNR: 22.86652946472168
Epoch 8, Loss: 0.0106192071042541, PSNR: 23.15807342529297
Epoch 9, Loss: 0.009325543082215719, PSNR: 23.963144302368164
Epoch 10, Loss: 0.008239106378621526, PSNR: 24.149322509765625
Epoch 11, Loss: 0.009789762520490007, PSNR: 23.948572158813477
Epoch 12, Loss: 0.007916494013948573, PSNR: 24.47007179260254
Epoch 13, Loss: 0.011024735357043229, PSNR: 23.800880432128906
Epoch 14, Loss: 0.009159617669259509, PSNR: 23.996694564819336
Epoch 15, Loss: 0.0068721249311541515, PSNR: 24.47078132629394

In [ ]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\Seconds_That_Count-1080p.mov", "data\Seconds_That_Count-native.mov", epoches=101, lr=0.001, max_frames=500, pretrained="saved_epoches/srcnn_epoch_28_norm_clip_1k.pth")

Begin Training
Epoch 0, Loss: 0.048391152359545234, PSNR: 14.647433280944824
Epoch 1, Loss: 0.039446291203300156, PSNR: 17.16032600402832
Epoch 2, Loss: 0.020805163697029155, PSNR: 19.50752830505371
Epoch 3, Loss: 0.01965670352284279, PSNR: 20.314931869506836
Epoch 4, Loss: 0.016798682083996635, PSNR: 21.186250686645508
Epoch 5, Loss: 0.013998423955248048, PSNR: 22.166027069091797
Epoch 6, Loss: 0.012076205153587379, PSNR: 22.633615493774414
Epoch 7, Loss: 0.01115076441783458, PSNR: 23.1796875
Epoch 8, Loss: 0.010351554522963448, PSNR: 23.53917694091797
Epoch 9, Loss: 0.01109986835702633, PSNR: 23.54061508178711
Epoch 10, Loss: 0.012444799552516391, PSNR: 23.028305053710938
Epoch 11, Loss: 0.007346106324500094, PSNR: 24.5755672454834
Epoch 12, Loss: 0.007170460904793193, PSNR: 24.892776489257812
Epoch 13, Loss: 0.006477111180882073, PSNR: 24.853708267211914
Epoch 14, Loss: 0.006820497633371916, PSNR: 24.916255950927734
Epoch 15, Loss: 0.00558759464830574, PSNR: 25.24536895751953
Epoch 

In [5]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\LifeUntouched_P3_4K_PQ_XQ-1080p.mov", "data\LifeUntouched_P3_4K_PQ_XQ-native.mov", epoches=101, lr=0.001, max_frames=500, pretrained="saved_epoches/srcnn_epoch_28_norm_clip_1k.pth")

Begin Training
Epoch 0, Loss: 0.035466157365590334, PSNR: 16.532346725463867
Epoch 1, Loss: 0.021385477907541726, PSNR: 20.07396125793457
Epoch 2, Loss: 0.01705504845206936, PSNR: 21.170434951782227
Epoch 3, Loss: 0.011852207127958536, PSNR: 21.590181350708008
Epoch 4, Loss: 0.008476512791174982, PSNR: 23.219743728637695
Epoch 5, Loss: 0.007540515960297651, PSNR: 23.67238998413086
Epoch 6, Loss: 0.004833338342400061, PSNR: 24.722068786621094
Epoch 7, Loss: 0.0036706862609005636, PSNR: 25.3326416015625
Epoch 8, Loss: 0.007431689210029112, PSNR: 24.023401260375977
Epoch 9, Loss: 0.018465844422785774, PSNR: 22.996623992919922
Epoch 10, Loss: 0.014018455572012398, PSNR: 20.657957077026367
Epoch 11, Loss: 0.011448215331054397, PSNR: 20.775583267211914
Epoch 12, Loss: 0.009323635138571262, PSNR: 21.660573959350586
Epoch 13, Loss: 0.007460980883075131, PSNR: 22.031803131103516
Epoch 14, Loss: 0.0064463009540405534, PSNR: 23.058504104614258
Epoch 15, Loss: 0.005020298503546251, PSNR: 23.930397

In [1]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\Lifting_Off-1080p.mov", "data\Lifting_Off-native.mov", epoches=101, lr=0.001, max_frames=500, pretrained="saved_epoches/srcnn_epoch_28_norm_clip_1k.pth")

Begin Training
Epoch 0, Loss: 0.03931137016043067, PSNR: 16.868621826171875
Epoch 1, Loss: 0.02551765243212382, PSNR: 18.23499870300293
Epoch 2, Loss: 0.03205111291673449, PSNR: 17.51097297668457
Epoch 3, Loss: 0.02286984326524867, PSNR: 18.41217803955078
Epoch 4, Loss: 0.02492249622527096, PSNR: 19.21213722229004
Epoch 5, Loss: 0.021801105855653685, PSNR: 19.34747314453125
Epoch 6, Loss: 0.023119889096253447, PSNR: 18.369112014770508
Epoch 7, Loss: 0.01734866781367196, PSNR: 19.524221420288086
Epoch 8, Loss: 0.01900467447315653, PSNR: 19.68594741821289
Epoch 9, Loss: 0.016107375992255077, PSNR: 20.06617546081543
Epoch 10, Loss: 0.015600854944851663, PSNR: 20.00078010559082
Epoch 11, Loss: 0.015219199729876386, PSNR: 20.360788345336914
Epoch 12, Loss: 0.01235029473900795, PSNR: 21.017440795898438
Epoch 13, Loss: 0.012845138346569406, PSNR: 20.3078670501709
Epoch 14, Loss: 0.012843242484248348, PSNR: 20.444503784179688
Epoch 15, Loss: 0.009857456975926956, PSNR: 21.2296085357666
Epoch 1

In [1]:
from models.cnn import trainSuperResCNN

trainSuperResCNN("data\Moment+of+Intensity-1080p.mov", "data\Moment+of+Intensity-native.mov", epoches=101, lr=0.001, max_frames=500, pretrained="saved_epoches/srcnn_epoch_28_norm_clip_1k.pth")

Begin Training
Epoch 0, Loss: 0.02439192174933851, PSNR: 20.20705223083496
Epoch 1, Loss: 0.012360782310780551, PSNR: 21.8241024017334
Epoch 2, Loss: 0.015589376652820243, PSNR: 21.650388717651367
Epoch 3, Loss: 0.017952796951350238, PSNR: 21.60489845275879
Epoch 4, Loss: 0.01865656099592646, PSNR: 20.029298782348633
Epoch 5, Loss: 0.01881096224921445, PSNR: 21.063241958618164
Epoch 6, Loss: 0.0181747755397939, PSNR: 20.305810928344727
Epoch 7, Loss: 0.020259689384450514, PSNR: 19.568117141723633
Epoch 8, Loss: 0.020389452240326338, PSNR: 20.693363189697266
Epoch 9, Loss: 0.019999015584795013, PSNR: 20.708616256713867
Epoch 10, Loss: 0.018290891519023314, PSNR: 21.204246520996094
Epoch 11, Loss: 0.0204163511387176, PSNR: 20.743793487548828
Epoch 12, Loss: 0.01919136188613872, PSNR: 20.982311248779297
Epoch 13, Loss: 0.019256265161352024, PSNR: 20.25801658630371
Epoch 14, Loss: 0.014942899915493198, PSNR: 21.664762496948242
Epoch 15, Loss: 0.012584157585580315, PSNR: 22.121910095214844


In [2]:
from models.cnn import CNNReconstructor

cnn_reconstructor = CNNReconstructor("srcnn_best.pth")
cnn_reconstructor.process_video("data\Lifting_Off-1080p.mov", "Lifting_Off_srcnn_1.mov")

3840 2160
Reconstructed video saved to Lifting_Off_srcnn_1.mov


In [11]:
from preprocess import downsample

video_path = "data\LifeUntouched_P3_4K_PQ_XQ-native-1080p.mov"
downsample(video_path, 500, target_width=1920 // 2, target_height=1080 // 2)


Finished processing video.


In [13]:
video_path = "data\Moment+of+Intensity-native-1080p.mov"
downsample(video_path, 500, target_width=1920 // 2, target_height=1080 // 2)
# downsample(video_path, 500, target_width=1920, target_height=1080)

Finished processing video.


### **CNN**

In [ ]:
from models.autoencoder import VideoAutoencoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VideoAutoencoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

video_dataset = VideoDataset(native_path, native_path, max_frames=50)
video_dataloader = DataLoader(video_dataset, batch_size=1, shuffle=True)

print("Begin Training")
for epoch in range(10):
    total_loss = 0
    for input_frames, target_frames in video_dataloader:
        input_frames, target_frames = input_frames.to(device), target_frames.to(device)

        optimizer.zero_grad()
        output = model(input_frames)
        loss = criterion(output, target_frames / 255)
        loss.backward()
        optimizer.step()

        del input_frames, target_frames, output
        torch.cuda.empty_cache()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(video_dataloader)}")

torch.save(model.state_dict(), "autoencoder.pth")
video_dataset.close()


In [ ]:
from models.autoencoder import VideoProcessor
model = VideoProcessor("autoencoder.pth")

model.encode_video(native_path, "encoded_video.mov")



In [ ]:
model.reconstruct_video("encoded_video.mov", "auto_reconstructed.mov")

In [ ]:
from models.cnn import CNNInterpolator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNInterpolator().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

video_dataset = VideoDataset(empty_checkerboard_path, native_path, max_frames=100)
video_dataloader = DataLoader(video_dataset, batch_size=1, shuffle=True)

print("Begin Training")
for epoch in range(10):
    total_loss = 0
    for input_frames, target_frames in video_dataloader:
        input_frames, target_frames = input_frames.to(device), target_frames.to(device)

        optimizer.zero_grad()
        output = model(input_frames)
        loss = criterion(output, target_frames)
        loss.backward()
        optimizer.step()

        del input_frames, target_frames, output
        torch.cuda.empty_cache()

        total_loss += loss.item()
        print(loss.item())
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(video_dataloader)}")

torch.save(model.state_dict(), "cnn_interpolator.pth")
video_dataset.close()


In [ ]:
from models.cnn import CNNReconstructor

reconstructor = CNNReconstructor("cnn_interpolator.pth")
reconstructor.process_video("data\LifeUntouched_P3_4K_PQ_XQ-checkerboard-empty.mov", "output_reconstructed.mov")

### **SRGAN**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from models.srgan import SRGANGenerator, SRGANDiscriminator, SRGANReconstructor
from torchvision import models as pretrained

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
lr_gen = 1e-4  # Learning rate for generator
lr_disc = 1e-4  # Learning rate for discriminator
num_epochs = 350
batch_size = 1
stride = 35

video_dataset = VideoDataset("data\Seconds_That_Count-540p.mov", "data\Seconds_That_Count-1080p.mov", max_frames=350, stride=stride)
video_dataloader = DataLoader(video_dataset, batch_size=batch_size, shuffle=True)

generator = SRGANGenerator().to(device)
discriminator = SRGANDiscriminator(img_H=1080, img_W=1920).to(device)

# Loss functions
gen_loss = nn.BCEWithLogitsLoss()
disc_loss = nn.BCEWithLogitsLoss()
mse_loss = nn.MSELoss()

# VGG for perceptual loss (if used)
vgg = pretrained.vgg19(pretrained=True).features[:7].to(device)
vgg.eval()
for param in vgg.parameters():
    param.requires_grad = False

# Optimizers
optimizer_gen = optim.Adam(generator.parameters(), lr=lr_gen, betas=(0.9, 0.999))
optimizer_disc = optim.Adam(discriminator.parameters(), lr=lr_disc, betas=(0.9, 0.999))

print("Begin Training")
for epoch in range(num_epochs):
    video_dataset.set_epoch(epoch % stride)
    adversarial_loss_total, al_count = 0, 0
    content_loss, cl_count = 0, 0
    for input_frames, target_frames in video_dataloader:
        input_frames = input_frames.to(device)
        target_frames = target_frames.to(device)

        # Train Discriminator
        discriminator.zero_grad()
        output_frames = generator(input_frames).detach()
        fake_label = discriminator(output_frames)
        real_label = discriminator(target_frames)

        df_loss = disc_loss(fake_label, torch.zeros_like(fake_label))
        dr_loss = disc_loss(real_label, torch.ones_like(real_label))
        adversarial_loss = df_loss + dr_loss
        adversarial_loss.backward()
        optimizer_disc.step()

        # Train Generator
        generator.zero_grad()
        output_frames = generator(input_frames)
        fake_label = discriminator(output_frames)
        g_loss = gen_loss(fake_label, torch.ones_like(fake_label))
        v_loss = mse_loss(vgg(output_frames), vgg(target_frames)) if vgg else 0
        m_loss = mse_loss(output_frames, target_frames)
        content_loss = g_loss + v_loss + m_loss
        content_loss.backward()
        optimizer_gen.step()

        del input_frames, target_frames, output_frames
        torch.cuda.empty_cache()
        adversarial_loss_total += adversarial_loss.item()
        al_count += 1
        cl_count += 1
        content_loss += content_loss.item()
        print(content_loss.item())

    print(f"Epoch {epoch+1}, Adv. Loss: {adversarial_loss_total / al_count}, Content Loss: {content_loss / cl_count}")
    
    if epoch % 35 == 0:
        torch.save(generator.state_dict(), f"srgan_generator_{epoch // 35}.pth")
        torch.save(discriminator.state_dict(), f"srgan_discriminator_{epoch // 35}.pth")

# Save final models
torch.save(generator.state_dict(), "srgan_generator.pth")
torch.save(discriminator.state_dict(), "srgan_discriminator.pth")
video_dataset.close()



c:\Users\harvi\miniconda3\envs\cs229\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\harvi\miniconda3\envs\cs229\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Begin Training
0.948185920715332
4.186858177185059
3.612298011779785
5.847818374633789
3.3046183586120605
3.065682888031006
2.9069435596466064
3.853748321533203
1.938880443572998
2.290120840072632
Epoch 1, Adv. Loss: 1.2284470915794372, Content Loss: 0.22901208698749542
2.500356912612915
2.4228196144104004
2.810717821121216
2.2759504318237305
1.6957778930664062
2.251575469970703
1.4278810024261475
2.0732710361480713
1.9991270303726196
Epoch 2, Adv. Loss: 1.2130202187432184, Content Loss: 0.22212523221969604
2.0558371543884277
1.2980657815933228
2.0368006229400635
2.053454875946045
2.0677690505981445
1.8876785039901733
2.0746991634368896
2.1790199279785156
1.6430597305297852
Epoch 3, Adv. Loss: 1.2169152365790472, Content Loss: 0.1825621873140335
2.2650883197784424
1.6241366863250732
1.3890790939331055
1.8076083660125732
1.5960031747817993
1.552962303161621
1.5661579370498657
2.6317782402038574
1.5561816692352295
Epoch 4, Adv. Loss: 1.5576243268118963, Content Loss: 0.17290908098220825


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from models.srgan import SRGANGenerator, SRGANDiscriminator, SRGANReconstructor
from torchvision import models as pretrained

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
lr_gen = 1e-4  # Learning rate for generator
lr_disc = 1e-4  # Learning rate for discriminator
num_epochs = 1000
batch_size = 1
stride = 35

video_dataset = VideoDataset("data\ImageDataset-540p.mov", "data\ImageDataset1080p.mov", max_frames=350, stride=stride)
video_dataloader = DataLoader(video_dataset, batch_size=batch_size, shuffle=True)

generator = SRGANGenerator().to(device)
discriminator = SRGANDiscriminator(img_H=1080, img_W=1920).to(device)

# Loss functions
gen_loss = nn.BCEWithLogitsLoss()
disc_loss = nn.BCEWithLogitsLoss()
mse_loss = nn.MSELoss()

# VGG for perceptual loss (if used)
vgg = pretrained.vgg19(pretrained=True).features[:7].to(device)
vgg.eval()
for param in vgg.parameters():
    param.requires_grad = False

# Optimizers
optimizer_gen = optim.Adam(generator.parameters(), lr=lr_gen, betas=(0.9, 0.999))
optimizer_disc = optim.Adam(discriminator.parameters(), lr=lr_disc, betas=(0.9, 0.999))

print("Begin Training")
for epoch in range(num_epochs):
    video_dataset.set_epoch(epoch % stride)
    adversarial_loss_total, al_count = 0, 0
    content_loss, cl_count = 0, 0
    for input_frames, target_frames in video_dataloader:
        input_frames = input_frames.to(device)
        target_frames = target_frames.to(device)

        # Train Discriminator
        discriminator.zero_grad()
        output_frames = generator(input_frames).detach()
        fake_label = discriminator(output_frames)
        real_label = discriminator(target_frames)

        df_loss = disc_loss(fake_label, torch.zeros_like(fake_label))
        dr_loss = disc_loss(real_label, torch.ones_like(real_label))
        adversarial_loss = df_loss + dr_loss
        adversarial_loss.backward()
        optimizer_disc.step()

        # Train Generator
        generator.zero_grad()
        output_frames = generator(input_frames)
        fake_label = discriminator(output_frames)
        g_loss = gen_loss(fake_label, torch.ones_like(fake_label))
        v_loss = mse_loss(vgg(output_frames), vgg(target_frames)) if vgg else 0
        m_loss = mse_loss(output_frames, target_frames)
        content_loss = g_loss + v_loss + m_loss
        content_loss.backward()
        optimizer_gen.step()

        del input_frames, target_frames, output_frames
        torch.cuda.empty_cache()
        adversarial_loss_total += adversarial_loss.item()
        al_count += 1
        cl_count += 1
        content_loss += content_loss.item()
        print(content_loss.item())

    print(f"Epoch {epoch+1}, Adv. Loss: {adversarial_loss_total / al_count}, Content Loss: {content_loss / cl_count}")
    
    if epoch % 35 == 0:
        torch.save(generator.state_dict(), f"srgan_generator_{epoch // 35}.pth")
        torch.save(discriminator.state_dict(), f"srgan_discriminator_{epoch // 35}.pth")

# Save final models
torch.save(generator.state_dict(), "srgan_generator.pth")
torch.save(discriminator.state_dict(), "srgan_discriminator.pth")
video_dataset.close()



Begin Training
5.072027206420898
7.471100807189941
3.665584087371826
1.780956506729126
2.946913003921509
3.329263687133789
3.425931930541992
2.4850282669067383
1.2974871397018433
3.080183982849121
Epoch 1, Adv. Loss: 1.4255594611167908, Content Loss: 0.30801841616630554
2.6191346645355225
3.151639699935913
2.8586490154266357
1.6845762729644775
2.5393881797790527
1.2956573963165283
3.3976011276245117
1.9861390590667725
1.8046839237213135
Epoch 2, Adv. Loss: 1.626523494720459, Content Loss: 0.20052044093608856
2.2366158962249756
2.804427146911621
2.506387710571289
2.234602689743042
1.7336798906326294
3.2808010578155518
1.3773096799850464
2.046682357788086
2.181661605834961
Epoch 3, Adv. Loss: 1.626523494720459, Content Loss: 0.24240684509277344
1.4029438495635986
1.7106050252914429
1.5466585159301758
2.4400458335876465
2.105806827545166
1.6408812999725342
0.96552574634552
1.0745073556900024
1.9115383625030518
Epoch 4, Adv. Loss: 1.626523494720459, Content Loss: 0.2123931497335434
2.35046

In [10]:
from models.srgan import SRGANReconstructor

reconstructor = SRGANReconstructor("srgan_generator.pth")
reconstructor.process_video("data\Seconds_That_Count-540p.mov", "output_reconstructed_SRGAN.mov", max_frames=100)

(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1

### **Tiled MAE**

In [1]:
from models.tiled_mae import process_video_mae

process_video_mae("data\Lifting_Off-native.mov", "Lifting_Off_mae.mov")

c:\Users\harvi\miniconda3\envs\cs229\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>
3840 2160
Reconstructed video saved to Lifting_Off_mae.mov


In [1]:
from models.tiled_mae import process_video_mae

process_video_mae("data\ImageDataset4k.mov", "ImageDataset4k_mae.mov")

c:\Users\harvi\miniconda3\envs\cs229\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>
3840 2160
Reconstructed video saved to ImageDataset4k_mae.mov


## **Evaluation**

In [11]:
from models.cnn import CNNReconstructor

cnn_reconstructor = CNNReconstructor("saved_epoches\srcnn_best_clip_seconds_that_count.pth")
cnn_reconstructor.process_video("data\Seconds_That_Count-1080p.mov", "data/srcnn/Seconds_That_Count_srcnn.mov")

3840 2160
Reconstructed video saved to data/srcnn/Seconds_That_Count_srcnn.mov


In [ ]:
from score import calculate_mse
calculate_mse("data\Seconds_That_Count-native.mov", "data/srcnn/Seconds_That_Count_srcnn.mov")

np.float32(6683258400.0)

In [9]:
from models.cnn import CNNReconstructor

cnn_reconstructor = CNNReconstructor("saved_epoches\srcnn_best_clip_life_untouched.pth")
cnn_reconstructor.process_video("data\LifeUntouched_P3_4K_PQ_XQ-1080p.mov", "data/srcnn/LifeUntouched_srcnn.mov")

3840 2160
Reconstructed video saved to data/srcnn/LifeUntouched_srcnn.mov


In [10]:
from score import calculate_mse
calculate_mse("data\LifeUntouched_P3_4K_PQ_XQ-native.mov", "data/srcnn/LifeUntouched_srcnn.mov")

np.float32(1511250700.0)

In [11]:
cnn_reconstructor = CNNReconstructor("saved_epoches\srcnn_best_clip_moments.pth")
cnn_reconstructor.process_video("data\Moment+of+Intensity-1080p.mov", "data/srcnn/Moments_srcnn.mov")

3840 2160
Reconstructed video saved to data/srcnn/Moments_srcnn.mov


In [12]:
calculate_mse("data\Moment+of+Intensity-native.mov", "data/srcnn/Moments_srcnn.mov")

np.float32(4927471000.0)

In [13]:
cnn_reconstructor = CNNReconstructor("saved_epoches\srcnn_best_clip_lifting_off.pth")
cnn_reconstructor.process_video("data\Lifting_Off-1080p.mov", "data/srcnn/LiftingOff_srcnn.mov")

3840 2160
Reconstructed video saved to data/srcnn/LiftingOff_srcnn.mov


In [14]:
calculate_mse("data\Lifting_Off-native.mov", "data/srcnn/LiftingOff_srcnn.mov")

np.float32(4513565000.0)

In [1]:
from models.srgan import SRGANReconstructor

reconstructor = SRGANReconstructor("srgan_generator.pth")
reconstructor.process_video("data\Seconds_That_Count-540p.mov", "data\srgan\Seconds_that_count_srgan.mov", max_frames=500)

Reconstructed 4K video saved to data\srgan\Seconds_that_count_srgan.mov


In [2]:
from score import calculate_mse
calculate_mse("data\Seconds_That_Count-1080p.mov", "data/srgan/Seconds_That_Count_srgan.mov")

np.float32(1354038900.0)

In [1]:
from models.srgan import SRGANReconstructor

reconstructor = SRGANReconstructor("srgan_generator.pth")
reconstructor.process_video("data\LifeUntouched_P3_4K_PQ_XQ-540p.mov", "data\srgan\LifeUntouched_srgan.mov", max_frames=500)

Reconstructed 4K video saved to data\srgan\LifeUntouched_srgan.mov


In [2]:
from models.srgan import SRGANReconstructor

reconstructor = SRGANReconstructor("srgan_generator.pth")
reconstructor.process_video("data\LifeUntouched_P3_4K_PQ_XQ-1080p.mov", "data\srgan\LifeUntouched_srgan_4k.mov", max_frames=500)

Reconstructed 4K video saved to data\srgan\LifeUntouched_srgan_4k.mov


In [ ]:
from score import calculate_mse
calculate_mse("data\LifeUntouched_P3_4K_PQ_XQ-1080p.mov", "data/srgan/LifeUntouched_srgan.mov")

np.float32(1508035500.0)

In [3]:
from score import calculate_mse
calculate_mse("data\LifeUntouched_P3_4K_PQ_XQ-native.mov", "data/srgan/LifeUntouched_srgan_4k.mov")

np.float32(5398398500.0)

In [ ]:
calculate_mse("data\LifeUntouched_P3_4K_PQ_XQ-1080p.mov", "data/srgan/LifeUntouched_srgan.mov")

In [3]:
from models.srgan import SRGANReconstructor

reconstructor = SRGANReconstructor("srgan_generator.pth")
reconstructor.process_video("data\Moment+of+Intensity-540p.mov", "data\srgan\Moment_srgan.mov", max_frames=500)

Reconstructed 4K video saved to data\srgan\Moment_srgan.mov


In [ ]:
from score import calculate_mse
calculate_mse("data\Moment+of+Intensity-1080p.mov", "data/srgan/Moment_srgan.mov")

np.float32(1864719700.0)

In [7]:
from models.srgan import SRGANReconstructor

reconstructor = SRGANReconstructor("srgan_generator.pth")
reconstructor.process_video("data\Lifting_Off-540p.mov", "data\srgan\LiftingOff_srgan.mov", max_frames=500)

Reconstructed 4K video saved to data\srgan\LiftingOff_srgan.mov


In [9]:
from score import calculate_mse
calculate_mse("data\Lifting_Off-1080p.mov", "data/srgan/LiftingOff_srgan.mov")

np.float32(1891619300.0)

## **Frame Evaluation**

In [7]:
from models.cnn import CNNReconstructor

cnn_reconstructor = CNNReconstructor("saved_epoches\srcnn_best_clip_seconds_that_count.pth")
cnn_reconstructor.process_image("final_report_images\\test-frame-1-1080p.jpg")

cnn_reconstructor = CNNReconstructor("saved_epoches\srcnn_best_clip_lifting_off.pth")
cnn_reconstructor.process_image("final_report_images\\test-frame-2-1080p.jpg")


Reconstructed image saved to final_report_images\test-frame-1-1080p_reconstructed.jpg
Reconstructed image saved to final_report_images\test-frame-2-1080p_reconstructed.jpg


'final_report_images\\test-frame-2-1080p_reconstructed.jpg'

In [1]:
from models.srgan import SRGANReconstructor

reconstructor = SRGANReconstructor("srgan_generator.pth")
reconstructor.process_image("final_report_images\\test-frame-1.jpg")
reconstructor.process_image("final_report_images\\test-frame-2.jpg")

Reconstructed image saved to final_report_images\test-frame-1_reconstructed_srgan.jpg
Reconstructed image saved to final_report_images\test-frame-2_reconstructed_srgan.jpg


'final_report_images\\test-frame-2_reconstructed_srgan.jpg'

In [5]:
from models.srgan import SRGANReconstructor

reconstructor = SRGANReconstructor("srgan_generator.pth")
reconstructor.process_image("final_report_images\\test-frame-1-1080p.jpg")
reconstructor.process_image("final_report_images\\test-frame-2-1080p.jpg")

Reconstructed image saved to final_report_images\test-frame-1-1080p_reconstructed_srgan.jpg
Reconstructed image saved to final_report_images\test-frame-2-1080p_reconstructed_srgan.jpg


'final_report_images\\test-frame-2-1080p_reconstructed_srgan.jpg'

In [ ]:
from models.tiled_mae import process_image_mae

process_image_mae("final_report_images\\test-frame-1.jpg")
process_image_mae("final_report_images\\test-frame-2.jpg")